# Rigid Bodies

In [ ]:
import warnings
warnings.simplefilter("ignore")

import os 

import foyer
import hoomd
from hoomd.data import make_snapshot
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import numpy as np
import matplotlib.pyplot as plt

from planckton.utils.rigid import connect_rings, moit, init_rigid
from planckton.utils.utils import set_coeffs
from planckton.init import Compound, Pack

# trying to modify planckton's init and sim functions

## TODO:
- write compound.visualize_rigid()
- figure out why some compounds work and some don't

In [ ]:
from planckton.compounds import COMPOUND_FILE                                   
from planckton.force_fields import FORCE_FIELD 
from planckton.sim import Simulation

In [ ]:
keys = list(COMPOUND_FILE.keys())
print(keys)

In [ ]:
for key in keys:
    try:
        comp = Compound(COMPOUND_FILE[key], rigid=True)
        compound=[comp]
        n_compounds=[2]

        packer = Pack(compound,n_compounds,density=0.01)
        #print(packer.rigid_typeids,packer.rigid_inds)
        typed_system = packer.pack()
        #print(*zip(packer.rigid_typeids,packer.rigid_inds),sep="\n")
        my_sim = Simulation(                                                    
            typed_system,                                                             
            kT=3.0,                                                             
            gsd_write=1e2,                                                     
            log_write=1e2,                                                                                                              
            rigid_inds=packer.rigid_inds, 
            rigid_typeids=packer.rigid_typeids,
            n_steps=3e3,                                                        
            mode="cpu",                                                         
            shrink_time=1e3,                                                    
        )                                                                       
        my_sim.run() 
        print("\n!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"{key} worked!")
        print("!!!!!!!!!!!!!!!!!!!!!!!\n")
    except RuntimeError:
        print("\n!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"{key} didn't work!")
        print("!!!!!!!!!!!!!!!!!!!!!!!\n")

# Working example with Rigid Bodies

Let's start with a simple case--a benzene molcule.

In [ ]:
bz_str = "c1ccccc1"
bz = mb.load(bz_str, smiles=True, ignore_box_warn=True)
name = "bz"
bz.name = name
#bz.visualize().show()

In [ ]:
#npt_str = "c1ccc2ccccc2c1"
#npt = mb.load(npt_str, smiles=True, ignore_box_warn=True)
#npt.visualize().show()

In [ ]:
#py_str = "c1cc2cccc3ccc4cccc1c4c32"
#py = mb.load(py_str, smiles=True, ignore_box_warn=True)
#py.visualize().show()

Let's try fixing the orientation of the benzene molecules when we fill the box--I think that might make it work better with rigid bodies.

In [ ]:
box = mb.Box([5,5,5])
system = mb.fill_box(bz, n_compounds=10, box=box, fix_orientation=True)
#system.visualize().show()

In [ ]:
gaff = foyer.forcefields.load_GAFF()
pmd_system = system.to_parmed(residues=[name])
typed_system = gaff.apply(system)
#print(set([atom.type for atom in typed_system.atoms]))


I'm using mbuild from [PR #808](https://github.com/mosdef-hub/mbuild/pull/808) which allows `create_hoomd_simulation` to read in a snapshot:

```
create_hoomd_simulation(structure, ref_distance=1.0, ref_mass=1.0, ref_energy=1.0, r_cut=1.2, auto_scale=False, snapshot_kwargs={}, pppm_kwargs={'Nx': 8, 'Ny': 8, 'Nz': 8, 'order': 4}, init_snap=None)
```

First, make a snapshot with 10 rigid particles--one for each benzene ring:

In [ ]:
sim = hoomd.context.SimulationContext()

with sim:
    hoomd.context.initialize("")
    init_snap = make_snapshot(N=10, particle_types=["_R"], box=hoomd.data.boxdim(L=10))

In [ ]:
with sim:
    hoomd_objects, ref_values = create_hoomd_simulation(
        typed_system, auto_scale=True, init_snap=init_snap
    )
    snap = hoomd_objects[0]

Want to do something with [ring detection](https://openbabel.org/wiki/Ring_detection) where rings are automatically converted to rigid bodies.

- convert to pybel mol
- use smarts matching to find rings
- make rigid

[SSSR documentation](http://openbabel.org/dev-api/classOpenBabel_1_1OBRing.shtml#_details)

In [ ]:
system_mol = system.to_pybel()
rings = sorted(connect_rings(system_mol), key=lambda x: x[0])

#print(*rings, sep="\n")
print(len(rings))

Now let's move the rigid body centers to the center of the ring and set the body IDs

In [ ]:
for i,ring in enumerate(rings):
    inds = ring + len(rings)
    snap.particles.position[i] = np.mean(snap.particles.position[inds], axis=0)
    snap.particles.body[i] = i
    snap.particles.body[inds] = i * np.ones(len(ring))

I am using [this example](http://farside.ph.utexas.edu/teaching/336k/Newtonhtml/node64.html) for how to calculate the moment of inertia tensor (also Matty/Mike/someone's code from cmeutils)

From [hoomd docs](https://hoomd-blue.readthedocs.io/en/v2.9.3/module-md-constrain.html#hoomd.md.constrain.rigid)
>The mass and moment of inertia of the central particle set the full mass and moment of inertia of the rigid body (constituent particle mass is ignored).

>The central particle is at the center of mass of the rigid body and the orientation quaternion defines the rotation from the body space into the simulation box. In body space, the center of mass of the body is at 0,0,0 and the moment of inertia is diagonal. 

In [ ]:
for i,ring in enumerate(rings):
    inds = ring + len(rings)
    snap.particles.moment_inertia[i] = moit(
        snap.particles.position[inds], snap.particles.mass[inds], center=snap.particles.position[i]
    )
    snap.particles.mass[i] = np.sum(snap.particles.mass[inds])

In [ ]:
# need to reinitialize
sim.system_definition.initializeFromSnapshot(snap)

In [ ]:
nl = sim.neighbor_lists[0]
ex_list = nl.exclusions 
ex_list.append('body')
sim.neighbor_lists[0].reset_exclusions(exclusions=ex_list)

print(sim.neighbor_lists[0].exclusions)

In [ ]:
with sim:
    rigid = hoomd.md.constrain.rigid()
    
    r_pos = snap.particles.position[0]
    const_pos = snap.particles.position[rings[0]+len(rings)]
    const_pos -= r_pos
    #print(r_pos,const_pos)
    
    const_types = [snap.particles.types[i] for i in snap.particles.typeid[rings[0]+len(rings)]]
    #print(const_types)
    
    rigid.set_param("_R", types=const_types, positions=[tuple(i) for i in const_pos])
    rigid.validate_bodies()
    
    lj = sim.forces[0]
    lj.pair_coeff.set("_R", snap.particles.types, epsilon=0, sigma=0)
    
    centers = hoomd.group.rigid_center()
    nonrigid = hoomd.group.nonrigid()
    _all = hoomd.group.all()
    
    hoomd.md.integrate.mode_standard(dt=0.0001);
    hoomd.md.integrate.langevin(group=centers, kT=1.0, seed=42);
    hoomd.md.integrate.langevin(group=nonrigid, kT=1.0, seed=42);
    hoomd.dump.gsd(filename="start.gsd", overwrite=True, period=None, group=_all, time_step=0)
    hoomd.dump.gsd("trajectory.gsd", period=1e3, group=_all, overwrite=True)
    hoomd.run(5e4)

In [ ]:
with sim:
    hoomd.dump.gsd(filename="after.gsd", overwrite=True, period=None, group=hoomd.group.all())